In [29]:
import sqlite3
from datetime import datetime, timedelta
import random

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.output_parsers.json import JsonOutputParser
from langchain_core.prompts.prompt import PromptTemplate

from langgraph.prebuilt import create_react_agent

In [30]:
# con = sqlite3.connect("agent.db")
# cur = con.cursor()

# # Criando a tabela sells (se não existir)
# cur.execute("""
# CREATE TABLE IF NOT EXISTS sells (
#     id INTEGER PRIMARY KEY,
#     product TEXT NOT NULL,
#     value REAL NOT NULL,
#     quantity INTEGER NOT NULL,
#     seller TEXT NOT NULL,
#     date TEXT NOT NULL
# )
# """)

# # Gerando datas aleatórias
# def random_date(start, end):
#     delta = end - start
#     random_days = random.randint(0, delta.days)
#     return start + timedelta(days=random_days)

# start_date = datetime(2023, 1, 1)
# end_date = datetime(2024, 1, 1)

# # Inserindo dados na tabela com datas aleatórias
# cur.execute("INSERT INTO sells (id, product, value, quantity, seller, date) VALUES (?, ?, ?, ?, ?, ?)",
#             (1, 'Smartphone', 1500.00, 2, 'João Assalim', random_date(start_date, end_date).strftime('%Y-%m-%d')))

# cur.execute("INSERT INTO sells (id, product, value, quantity, seller, date) VALUES (?, ?, ?, ?, ?, ?)",
#             (2, 'Notebook', 3500.00, 1, 'João Assalim', random_date(start_date, end_date).strftime('%Y-%m-%d')))

# cur.execute("INSERT INTO sells (id, product, value, quantity, seller, date) VALUES (?, ?, ?, ?, ?, ?)",
#             (3, 'Câmera DSLR', 2200.00, 1, 'Maria Silva', random_date(start_date, end_date).strftime('%Y-%m-%d')))

# cur.execute("INSERT INTO sells (id, product, value, quantity, seller, date) VALUES (?, ?, ?, ?, ?, ?)",
#             (4, 'Fone de ouvido', 200.00, 5, 'Carlos Almeida', random_date(start_date, end_date).strftime('%Y-%m-%d')))

# cur.execute("INSERT INTO sells (id, product, value, quantity, seller, date) VALUES (?, ?, ?, ?, ?, ?)",
#             (5, 'Monitor', 800.00, 3, 'Ana Souza', random_date(start_date, end_date).strftime('%Y-%m-%d')))

# # Confirmando as alterações no banco de dados
# con.commit()

# # Verificando os dados inseridos
# cur.execute("SELECT * FROM sells")
# data = cur.fetchall()
# for row in data:
#     print(row)

# # Fechando a conexão com o banco de dados
# con.close()

In [31]:
con = sqlite3.connect("agent.db")
cur = con.cursor()
cur.execute("SELECT * FROM sells")
data = cur.fetchall()

for row in data:
    print(row)

(1, 'Smartphone', 1500.0, 2, 'João Assalim', '2023-10-23')
(2, 'Notebook', 3500.0, 1, 'João Assalim', '2023-09-19')
(3, 'Câmera DSLR', 2200.0, 1, 'Maria Silva', '2023-03-18')
(4, 'Fone de ouvido', 200.0, 5, 'Carlos Almeida', '2023-12-20')
(5, 'Monitor', 800.0, 3, 'Ana Souza', '2023-12-28')


In [32]:
llm = ChatGroq(model="llama3-70b-8192")

In [33]:
# construir agentes para realizar constultas no banco de dados
# produzir um relario baseado nessa consulta no bd

# serão algumas consultas.
#   exibir todos as tabelas
#   com base nas tabelas, colocar como contexto para um proximo agente
#   com base nas tabelas, agente decide em qual tabela fazer a consulta
#   gerar um relatorio com as informações retiradas

In [34]:
@tool
def get_context_on_web(question):
    """Use this to search on web about the user question if you cannot get any relevant context from database"""
    web_search_tool = TavilySearchResults(max_results=3)
    search_results = web_search_tool.invoke({"query": question})
        
    return "\n".join([result["content"] for result in search_results])


@tool
def get_information_from_database(question):
    """Use this to get information on database and create a report to the user"""
    tables = get_tables_name_in_database()
    agent_table_choice = define_which_table_uses(tables, question)
    agent_table_choice = agent_table_choice["agent_table_choice"]
    
    table_columns = get_table_structure(agent_table_choice)
    agent_query = create_database_query(agent_table_choice, table_columns, question)
    agent_query = agent_query["agent_sqlite_query"]
    
    print(agent_query)

    con = sqlite3.connect("agent.db")
    cur = con.cursor()
    queryset = [i for i in cur.execute(agent_query)]
    
    queryset_summarized = agent_summarizer_query(question, table_columns, str(queryset))
    
    return queryset_summarized["agent_insights"]



def get_table_structure(table):
    con = sqlite3.connect("agent.db")
    cur = con.cursor()
    data = cur.execute(f"SELECT * FROM {table}")
    
    table_columns = ", ".join([column[0] for column in data.description])
    return table_columns
    

def get_tables_name_in_database():
    con = sqlite3.connect("agent.db")
    cur = con.cursor()
    tables = ", ".join([i[0] for i in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table';")])
    return tables

def define_which_table_uses(tables, message):
    prompt = PromptTemplate(
        template="""
        You are an expert decide which table uses to make an query based on a user message. \n
        
        Below you will have all tables separated by ',' and you have to decide which table make an query.
        Always try to be clear and concise, and avoid adding unnecessary information.
        
        JUST ANSWER THE TABLE NAME, without any preamble or explaination and return the a JSON with a single key 'agent_table_choice'. \n\n
        
        MESSAGE: {message} \n\n
        TABLES NAMES: {tables}""",
        
        input_variables=[
            "message",
            "tables",
        ],
    )
    
    table_agent = prompt | llm | JsonOutputParser()
    return table_agent.invoke({"message": message, "tables": tables})

def create_database_query(tables, columns, message):
    prompt = PromptTemplate(
        template="""
        You are an expert at creating SQLite queries to execute with Python.

        Below, you have all the columns from the tables, separated by ',', and your task is to decide which columns to filter based on the user message.
        Try to understand the columns in the context of their table names to create a concise and useful query for future dashboard functions.
        If you don't find all the necessary columns to apply a filter, ignore the requested filter and proceed with the rest.
        ONLY use the columns that are provided to you, and DO NOT create or use any new columns.
        JUST use the folowing table columns to make this query: {columns}.

        Provide ONLY the SQLite query as a JSON object with a single key 'agent_sqlite_query', and no additional explanations.

        MESSAGE: {message} \n\n
        TABLE NAMES: {tables}""",
        input_variables=[
            "message",
            "tables",
            "columns"
        ],
    )
    
    table_agent = prompt | llm | JsonOutputParser()
    return table_agent.invoke({"message": message, "tables": tables, "columns": columns})

def agent_summarizer_query(message, table_columns, queryset):
    prompt = PromptTemplate(
        template="""
        You are an expert to summarize information from a queryset based on table columns and user message. \n
        
        Below you will have all columns from table separated by ',' and the user message.
        based on the queryset bellow, try to make insights to the user:
        
        QUERYSET: {queryset}
        
        Make insights and return the a JSON with a single key 'agent_insights'. \n\n
        
        MESSAGE: {message} \n\n
        TABLES COLUMNS: {table_columns}""",
        
        input_variables=[
            "queryset",
            "message",
            "table_columns",
        ],
    )
    
    table_agent = prompt | llm | JsonOutputParser()
    return table_agent.invoke({"queryset": queryset, "message": message, "table_columns": table_columns})

In [35]:
tools = [get_information_from_database, get_context_on_web]

In [36]:
agent = create_react_agent(llm, tools=tools)

In [37]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [43]:
inputs = {"messages": [("user", "make an dashboard about João Assalim sells last 2 years")]}
print_stream(agent.stream(inputs, stream_mode="values"))

================================ Human Message =================================

make an dashboard about João Assalim sells last 2 years
================================== Ai Message ==================================
Tool Calls:
  get_information_from_database (call_j64f)
 Call ID: call_j64f
  Args:
    question: João Assalim sells last 2 years
SELECT * FROM sells WHERE seller = 'João Assalim' AND date >= date('now', '-2 years')
================================= Tool Message =================================
Name: get_information_from_database

{"total_sales_value": 5000.0, "number_of_products_sold": 3, "products_sold": ["Smartphone", "Notebook"]}
================================== Ai Message ==================================

Based on the result, I'll respond directly without using a tool.

Here is a dashboard about João Assalim's sales over the last 2 years:

**Total Sales Value:** R$ 5000.00

**Number of Products Sold:** 3

**Products Sold:**

* Smartphone
* Notebook

Let me know

In [42]:
con = sqlite3.connect("agent.db")
cur = con.cursor()
data = cur.execute("SELECT * FROM sells WHERE seller = 'João Assalim' AND date > date('now', '-2 year');")

for i in data:
    print(i)

(1, 'Smartphone', 1500.0, 2, 'João Assalim', '2023-10-23')
(2, 'Notebook', 3500.0, 1, 'João Assalim', '2023-09-19')
